### Import Statements

In [ ]:
import tensorflow as tf
from keras import backend, callbacks
from keras.models import Model, load_model
from keras.layers import add, Add, BatchNormalization, Concatenate, Convolution1D, Dense, Dot, Dropout, Embedding, Input, Lambda, Layer, LayerNormalization, Permute, RepeatVector, Softmax, TimeDistributed
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

import math
import numpy as np
import sys
import os
from collections import OrderedDict
import ast
from scipy.special import softmax
import pandas as pd
import time

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

### Performance Evaluation Methods

#### 8-state (Q8) Secondary Structure Prediction

In [ ]:
def custom_categorical_cross_entropy(y_true, y_predicted):
    mask = backend.sum(y_true, axis=2)

    loss = backend.sum(y_true * backend.log(y_predicted + sys.float_info.epsilon), axis=2)
    loss = backend.sum(loss * mask, axis=1)

    return -1 * backend.sum(loss, axis=0)

def average_accuracy(y_true, y_predicted):
    mask = backend.sum(y_true, axis=2)

    y_true_labels, y_predicted_labels = backend.cast(backend.argmax(y_true, axis=2), "int8"), backend.cast(backend.argmax(y_predicted, axis=2), "int8")

    is_identical = backend.cast(backend.equal(y_true_labels, y_predicted_labels), "float32")
    num_identicals, protein_lengths = backend.sum(is_identical * mask, axis=1), backend.sum(mask, axis=1)

    return backend.mean(num_identicals / protein_lengths, axis=0)

def total_accuracy(y_true, y_predicted):
    mask = backend.sum(y_true, axis=2)

    y_true_labels, y_predicted_labels = backend.cast(backend.argmax(y_true, axis=2), "int8"), backend.cast(backend.argmax(y_predicted, axis=2), "int8")

    is_identical = backend.cast(backend.equal(y_true_labels, y_predicted_labels), "float32")
    num_identicals, protein_lengths = backend.sum(is_identical * mask, axis=1), backend.sum(mask, axis=1)

    return backend.sum(num_identicals, axis=0) / backend.sum(protein_lengths, axis=0)

def total_correct_prediction(y_true, y_predicted):
    mask = backend.sum(y_true, axis=2)

    y_true_labels, y_predicted_labels = backend.cast(backend.argmax(y_true, axis=2), "int8"), backend.cast(backend.argmax(y_predicted, axis=2), "int8")

    is_identical = backend.cast(backend.equal(y_true_labels, y_predicted_labels), "float32")
    num_identicals = backend.sum(is_identical * mask, axis=1)
    
    return backend.sum(num_identicals, axis=0)

#### Backbone Torsion φ and ψ Angles Prediction

**Performance Metrics:-**
- `tse` -> *Total Squared Error (TSE)*
- `mse` -> *Mean Squared Error (MSE)*
- `mae` -> *Mean Absolute Error (MAE)*
- `sae` -> *Sum of Absolute Error (SAE)* = *MAE* * `total_residue_count`

In [ ]:
def total_tse(y_true, y_predicted):
    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    
    y_true_phi_sine, y_true_phi_cosine = y_true[:, :, 0] * mask, y_true[:, :, 1] * mask
    y_true_psi_sine, y_true_psi_cosine = y_true[:, :, 2] * mask, y_true[:, :, 3] * mask
    y_pred_phi_sine, y_pred_phi_cosine = y_predicted[:, :, 0] * mask, y_predicted[:, :, 1] * mask
    y_pred_psi_sine, y_pred_psi_cosine = y_predicted[:, :, 2] * mask, y_predicted[:, :, 3] * mask
    
    phi_diff_sine, phi_diff_cosine = backend.abs(y_true_phi_sine - y_pred_phi_sine), backend.abs(y_true_phi_cosine - y_pred_phi_cosine)
    psi_diff_sine, psi_diff_cosine = backend.abs(y_true_psi_sine - y_pred_psi_sine), backend.abs(y_true_psi_cosine - y_pred_psi_cosine)
    phi_mse_sine, phi_mse_cosine = backend.sum(backend.square(phi_diff_sine)), backend.sum(backend.square(phi_diff_cosine))
    psi_mse_sine, psi_mse_cosine = backend.sum(backend.square(psi_diff_sine)), backend.sum(backend.square(psi_diff_cosine))
    
    return phi_mse_sine + phi_mse_cosine + psi_mse_sine + psi_mse_cosine

def mean_tse(y_true, y_predicted):
    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    
    y_true_phi_sine, y_true_phi_cosine = y_true[:, :, 0] * mask, y_true[:, :, 1] * mask
    y_true_psi_sine, y_true_psi_cosine = y_true[:, :, 2] * mask, y_true[:, :, 3] * mask
    y_pred_phi_sine, y_pred_phi_cosine = y_predicted[:, :, 0] * mask, y_predicted[:, :, 1] * mask
    y_pred_psi_sine, y_pred_psi_cosine = y_predicted[:, :, 2] * mask, y_predicted[:, :, 3] * mask
    
    phi_diff_sine, phi_diff_cosine = backend.abs(y_true_phi_sine - y_pred_phi_sine), backend.abs(y_true_phi_cosine - y_pred_phi_cosine)
    psi_diff_sine, psi_diff_cosine = backend.abs(y_true_psi_sine - y_pred_psi_sine), backend.abs(y_true_psi_cosine - y_pred_psi_cosine)
    phi_mse_sine, phi_mse_cosine = backend.sum(backend.square(phi_diff_sine)), backend.sum(backend.square(phi_diff_cosine))
    psi_mse_sine, psi_mse_cosine = backend.sum(backend.square(psi_diff_sine)), backend.sum(backend.square(psi_diff_cosine))
    
    return 0.25 * (phi_mse_sine + phi_mse_cosine + psi_mse_sine + psi_mse_cosine)

def total_mse(y_true, y_predicted):
    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    total_residue_count = backend.sum(mask)

    y_true_phi_sine, y_true_phi_cosine = y_true[:, :, 0] * mask, y_true[:, :, 1] * mask
    y_true_psi_sine, y_true_psi_cosine = y_true[:, :, 2] * mask, y_true[:, :, 3] * mask
    y_pred_phi_sine, y_pred_phi_cosine = y_predicted[:, :, 0] * mask, y_predicted[:, :, 1] * mask
    y_pred_psi_sine, y_pred_psi_cosine = y_predicted[:, :, 2] * mask, y_predicted[:, :, 3] * mask

    phi_diff_sine, phi_diff_cosine = backend.abs(y_true_phi_sine - y_pred_phi_sine), backend.abs(y_true_phi_cosine - y_pred_phi_cosine)
    psi_diff_sine, psi_diff_cosine = backend.abs(y_true_psi_sine - y_pred_psi_sine), backend.abs(y_true_psi_cosine - y_pred_psi_cosine)
    phi_mse_sine, phi_mse_cosine = backend.sum(backend.square(phi_diff_sine)) / total_residue_count, backend.sum(backend.square(phi_diff_cosine)) / total_residue_count
    psi_mse_sine, psi_mse_cosine = backend.sum(backend.square(psi_diff_sine)) / total_residue_count, backend.sum(backend.square(psi_diff_cosine)) / total_residue_count

    total_mse = phi_mse_sine + phi_mse_cosine + psi_mse_sine + psi_mse_cosine
    return total_mse

def mean_mse(y_true, y_predicted):
    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    total_residue_count = backend.sum(mask)

    y_true_phi_sine, y_true_phi_cosine = y_true[:, :, 0] * mask, y_true[:, :, 1] * mask
    y_true_psi_sine, y_true_psi_cosine = y_true[:, :, 2] * mask, y_true[:, :, 3] * mask
    y_pred_phi_sine, y_pred_phi_cosine = y_predicted[:, :, 0] * mask, y_predicted[:, :, 1] * mask
    y_pred_psi_sine, y_pred_psi_cosine = y_predicted[:, :, 2] * mask, y_predicted[:, :, 3] * mask

    phi_diff_sine, phi_diff_cosine = backend.abs(y_true_phi_sine - y_pred_phi_sine), backend.abs(y_true_phi_cosine - y_pred_phi_cosine)
    psi_diff_sine, psi_diff_cosine = backend.abs(y_true_psi_sine - y_pred_psi_sine), backend.abs(y_true_psi_cosine - y_pred_psi_cosine)
    phi_mse_sine, phi_mse_cosine = backend.sum(backend.square(phi_diff_sine)) / total_residue_count, backend.sum(backend.square(phi_diff_cosine)) / total_residue_count
    psi_mse_sine, psi_mse_cosine = backend.sum(backend.square(psi_diff_sine)) / total_residue_count, backend.sum(backend.square(psi_diff_cosine)) / total_residue_count

    mean_mse = 0.25 * (phi_mse_sine + phi_mse_cosine + psi_mse_sine + psi_mse_cosine)
    return mean_mse

def mean_mae(y_true, y_predicted):
    y_true_phi_angle = tf.atan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = tf.atan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi
    y_true_psi_angle = tf.atan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = tf.atan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi

    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    total_residue_count = backend.sum(mask)

    phi_diff, psi_diff = backend.abs(y_true_phi_angle - y_pred_phi_angle), backend.abs(y_true_psi_angle - y_pred_psi_angle)
    phi_diff_rev, psi_diff_rev = Lambda(lambda x: 360 - x)(phi_diff), Lambda(lambda x: 360 - x)(psi_diff)

    phi_mask = backend.cast(backend.greater(phi_diff[:, :], 180), dtype="float32")
    phi_mask_rev = 1 - phi_mask
    psi_mask = backend.cast(backend.greater(psi_diff[:, :], 180), dtype="float32")
    psi_mask_rev = 1 - psi_mask

    phi_error, psi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask, psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    phi_mae, psi_mae = backend.sum(phi_error * mask) / total_residue_count, backend.sum(psi_error * mask) / total_residue_count

    mean_mae = 0.5 * (phi_mae + psi_mae)
    return mean_mae

def phi_mae(y_true, y_predicted):
    y_true_phi_angle = tf.atan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = tf.atan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi

    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    total_residue_count = backend.sum(mask)

    phi_diff = backend.abs(y_true_phi_angle - y_pred_phi_angle)
    phi_diff_rev = Lambda(lambda x: 360 - x)(phi_diff)

    phi_mask = backend.cast(backend.greater(phi_diff[:, :], 180), dtype="float32")
    phi_mask_rev = 1 - phi_mask

    phi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask
    phi_mae = backend.sum(phi_error * mask) / total_residue_count
    return phi_mae

def psi_mae(y_true, y_predicted):
    y_true_psi_angle = tf.atan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = tf.atan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi
    
    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    total_residue_count = backend.sum(mask)
    
    psi_diff = backend.abs(y_true_psi_angle - y_pred_psi_angle)
    psi_diff_rev = Lambda(lambda x: 360 - x)(psi_diff)
    
    psi_mask = backend.cast(backend.greater(psi_diff[:, :], 180), dtype="float32")
    psi_mask_rev = 1 - psi_mask

    psi_error = psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    psi_mae = backend.sum(psi_error * mask) / total_residue_count
    return psi_mae

def mean_sae(y_true, y_predicted):
    y_true_phi_angle = tf.atan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = tf.atan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi
    y_true_psi_angle = tf.atan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = tf.atan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi

    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")

    phi_diff, psi_diff = backend.abs(y_true_phi_angle - y_pred_phi_angle), backend.abs(y_true_psi_angle - y_pred_psi_angle)
    phi_diff_rev, psi_diff_rev = Lambda(lambda x: 360 - x)(phi_diff), Lambda(lambda x: 360 - x)(psi_diff)

    phi_mask = backend.cast(backend.greater(phi_diff[:, :], 180), dtype="float32")
    phi_mask_rev = 1 - phi_mask
    psi_mask = backend.cast(backend.greater(psi_diff[:, :], 180), dtype="float32")
    psi_mask_rev = 1 - psi_mask

    phi_error, psi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask, psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    phi_sae, psi_sae = backend.sum(phi_error * mask), backend.sum(psi_error * mask)
    
    mean_sae = 0.5 * (phi_sae + psi_sae)
    return mean_sae

def phi_sae(y_true, y_predicted):
    y_true_phi_angle = tf.atan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = tf.atan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi
    
    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")
    
    phi_diff = backend.abs(y_true_phi_angle - y_pred_phi_angle)
    phi_diff_rev = Lambda(lambda x: 360 - x)(phi_diff)

    phi_mask = backend.cast(backend.greater(phi_diff[:, :], 180), dtype="float32")
    phi_mask_rev = 1 - phi_mask

    phi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask
    phi_sae = backend.sum(phi_error * mask)
    return phi_sae

def psi_sae(y_true, y_predicted):
    y_true_psi_angle = tf.atan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = tf.atan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi

    mask = 1 - backend.cast(backend.equal(y_true[:, :, 0], -500), dtype="float32")

    psi_diff = backend.abs(y_true_psi_angle - y_pred_psi_angle)
    psi_diff_rev = Lambda(lambda x: 360 - x)(psi_diff)

    psi_mask = backend.cast(backend.greater(psi_diff[:, :], 180), dtype="float32")
    psi_mask_rev = 1 - psi_mask

    psi_error = psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    psi_sae = backend.sum(psi_error * mask)
    return psi_sae

### `NumPy` Implementation of Performance Evaluation Methods

#### 8-state (Q8) Secondary Structure Prediction

In [ ]:
def average_accuracy_numpy(y_true, y_predicted):
    mask = np.sum(y_true, axis=2)
    
    y_true_labels, y_predicted_labels = np.argmax(y_true, axis=2).astype(np.int8), np.argmax(y_predicted, axis=2).astype(np.int8)
    
    is_identical = np.equal(y_true_labels, y_predicted_labels).astype(np.float32)
    num_identicals, protein_lengths = np.sum(is_identical * mask, axis=1), np.sum(mask, axis=1)
    
    return np.mean(num_identicals / protein_lengths, axis=0)

def total_accuracy_numpy(y_true, y_predicted):
    mask = np.sum(y_true, axis=2)
    
    y_true_labels, y_predicted_labels = np.argmax(y_true, axis=2).astype(np.int8), np.argmax(y_predicted, axis=2).astype(np.int8)
    
    is_identical = np.equal(y_true_labels, y_predicted_labels).astype(np.float32)
    num_identicals, protein_lengths = np.sum(is_identical * mask, axis=1), np.sum(mask, axis=1)
    
    return np.sum(num_identicals, axis=0) / np.sum(protein_lengths, axis=0)

def total_correct_prediction_numpy(y_true, y_predicted):
    mask = np.sum(y_true, axis=2)
    
    y_true_labels, y_predicted_labels = np.argmax(y_true, axis=2).astype(np.int8), np.argmax(y_predicted, axis=2).astype(np.int8)
    
    is_identical = np.equal(y_true_labels, y_predicted_labels).astype(np.float32)
    num_identicals = np.sum(is_identical * mask, axis=1)
    
    return np.sum(num_identicals, axis=0)

#### Backbone Torsion φ and ψ Angles Prediction

**Performance Metrics:-**
- `mae` -> *Mean Absolute Error (MAE)*

In [ ]:
def mean_mae_numpy_train(y_true, y_predicted):
    y_true_phi_angle = np.arctan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = np.arctan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi
    y_true_psi_angle = np.arctan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = np.arctan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi
    
    mask = 1 - np.equal(y_true[:, :, 0], -500).astype(np.float32)
    total_residue_count = np.sum(mask)
    
    phi_diff, psi_diff = np.abs(y_true_phi_angle - y_pred_phi_angle), np.abs(y_true_psi_angle - y_pred_psi_angle)
    
    revert_difference = lambda x: 360 - x
    phi_diff_rev, psi_diff_rev = revert_difference(phi_diff), revert_difference(psi_diff)
    
    phi_mask = np.greater(phi_diff[:, :], 180).astype(np.float32)
    phi_mask_rev = 1 - phi_mask
    psi_mask = np.greater(psi_diff[:, :], 180).astype(np.float32)
    psi_mask_rev = 1 - psi_mask
    
    phi_error, psi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask, psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    phi_mae, psi_mae = np.sum(phi_error * mask) / total_residue_count, np.sum(psi_error * mask) / total_residue_count
    
    mean_mae = 0.5 * (phi_mae + psi_mae)
    return mean_mae

def phi_mae_numpy_train(y_true, y_predicted):
    y_true_phi_angle = np.arctan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = np.arctan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi
    
    mask = 1 - np.equal(y_true[:, :, 0], -500).astype(np.float32)
    total_residue_count = np.sum(mask)
    
    phi_diff = np.abs(y_true_phi_angle - y_pred_phi_angle)
    
    revert_difference = lambda x: 360 - x
    phi_diff_rev = revert_difference(phi_diff)
    
    phi_mask = np.greater(phi_diff[:, :], 180).astype(np.float32)
    phi_mask_rev = 1 - phi_mask
    
    phi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask
    phi_mae = np.sum(phi_error * mask) / total_residue_count
    
    return phi_mae

def psi_mae_numpy_train(y_true, y_predicted):
    y_true_psi_angle = np.arctan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = np.arctan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi
    
    mask = 1 - np.equal(y_true[:, :, 0], -500).astype(np.float32)
    total_residue_count = np.sum(mask)
    
    psi_diff = np.abs(y_true_psi_angle - y_pred_psi_angle)
    
    revert_difference = lambda x: 360 - x
    psi_diff_rev = revert_difference(psi_diff)
    
    psi_mask = np.greater(psi_diff[:, :], 180).astype(np.float32)
    psi_mask_rev = 1 - psi_mask
    
    psi_error = psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    psi_mae = np.sum(psi_error * mask) / total_residue_count
    
    return psi_mae

In [ ]:
def mean_mae_numpy_test(y_true, y_predicted):
    y_true_phi_angle = np.arctan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = np.arctan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi
    y_true_psi_angle = np.arctan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = np.arctan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi
    
    phi_angle_mask = 1 - np.equal(y_true[:, :, 0], -500).astype(np.float32)
    psi_angle_mask = 1 - np.equal(y_true[:, :, 2], -500).astype(np.float32)
    phi_total_residue_count = np.sum(phi_angle_mask)
    psi_total_residue_count = np.sum(psi_angle_mask)
    
    phi_diff, psi_diff = np.abs(y_true_phi_angle - y_pred_phi_angle), np.abs(y_true_psi_angle - y_pred_psi_angle)
    
    revert_difference = lambda x: 360 - x
    phi_diff_rev, psi_diff_rev = revert_difference(phi_diff), revert_difference(psi_diff)
    
    phi_mask = np.greater(phi_diff[:, :], 180).astype(np.float32)
    phi_mask_rev = 1 - phi_mask
    psi_mask = np.greater(psi_diff[:, :], 180).astype(np.float32)
    psi_mask_rev = 1 - psi_mask
    
    phi_error, psi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask, psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    phi_mae, psi_mae = np.sum(phi_error * phi_angle_mask) / phi_total_residue_count, np.sum(psi_error * psi_angle_mask) / psi_total_residue_count
    
    mean_mae = 0.5 * (phi_mae + psi_mae)
    return mean_mae

def phi_mae_numpy_test(y_true, y_predicted):
    y_true_phi_angle = np.arctan2(y_true[:, :, 0], y_true[:, :, 1]) * 180 / np.pi
    y_pred_phi_angle = np.arctan2(y_predicted[:, :, 0], y_predicted[:, :, 1]) * 180 / np.pi
    
    phi_angle_mask = 1 - np.equal(y_true[:, :, 0], -500).astype(np.float32)
    phi_total_residue_count = np.sum(phi_angle_mask)
    
    phi_diff = np.abs(y_true_phi_angle - y_pred_phi_angle)
    
    revert_difference = lambda x: 360 - x
    phi_diff_rev = revert_difference(phi_diff)
    
    phi_mask = np.greater(phi_diff[:, :], 180).astype(np.float32)
    phi_mask_rev = 1 - phi_mask
    
    phi_error = phi_diff * phi_mask_rev + phi_diff_rev * phi_mask
    phi_mae = np.sum(phi_error * phi_angle_mask) / phi_total_residue_count
    
    return phi_mae

def psi_mae_numpy_test(y_true, y_predicted):
    y_true_psi_angle = np.arctan2(y_true[:, :, 2], y_true[:, :, 3]) * 180 / np.pi
    y_pred_psi_angle = np.arctan2(y_predicted[:, :, 2], y_predicted[:, :, 3]) * 180 / np.pi
    
    psi_angle_mask = 1 - np.equal(y_true[:, :, 2], -500).astype(np.float32)
    psi_total_residue_count = np.sum(psi_angle_mask)
    
    psi_diff = np.abs(y_true_psi_angle - y_pred_psi_angle)
    
    revert_difference = lambda x: 360 - x
    psi_diff_rev = revert_difference(psi_diff)
    
    psi_mask = np.greater(psi_diff[:, :], 180).astype(np.float32)
    psi_mask_rev = 1 - psi_mask
    
    psi_error = psi_diff * psi_mask_rev + psi_diff_rev * psi_mask
    psi_mae = np.sum(psi_error * psi_angle_mask) / psi_total_residue_count
    
    return psi_mae

### Utility Classes and Methods for Inference

In [ ]:
def get_shape_list(x):
    if backend.backend() != "theano":
        temp = backend.int_shape(x)
    else:
        temp = x.shape

    temp = list(temp)
    temp[0] = -1
    return temp

class CustomLayer(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._x = None
    
    def build(self, input_shape):
        self._x = backend.variable(0.2)
        self._x._trainable = True
        self._trainable_weights = [self._x]
        super().build(input_shape)

    def call(self, x, **kwargs):
        output_after_attention, previous_layer_input = x
        result = add([self._x * output_after_attention, (1 - self._x) * previous_layer_input])
        return result

    def compute_output_shape(self, input_shape):
        return input_shape[0]

class TestDataLoader(tf.keras.utils.Sequence):
    def __init__(self, dataset_path, dataset_name, features, num_features, batch_size):
        self.dataset_path, self.dataset_name = dataset_path, dataset_name
        self.features, self.num_features, self.batch_size = features, num_features, batch_size
        self.proteins_dict, self.protein_names = OrderedDict(), None
        
        with open(dataset_path + os.sep + dataset_name + "_below_700_proteins.txt", 'r') as proteins_file:
            for content in proteins_file.read().split('\n'):
                if content != '':
                    protein_name, protein_length = content.split(',')
                    self.proteins_dict[protein_name] = int(protein_length)
        
        self.protein_names = list(self.proteins_dict.keys())
    
    def __len__(self):
        return math.ceil(len(self.protein_names) / self.batch_size)
    
    def __getitem__(self, index):
        batch_protein_names = self.protein_names[index * self.batch_size:(index + 1) * self.batch_size]
        
        main_input = np.zeros(shape=(len(batch_protein_names), 700, self.num_features))
        attention_mask = np.zeros(shape=(len(batch_protein_names), 700))
        position_ids = np.zeros(shape=(len(batch_protein_names), 700))
        weight_mask = np.ones(shape=(len(batch_protein_names), 700))
        Q8_labels = np.zeros(shape=(len(batch_protein_names), 700, 8))
        phi_psi_labels = np.zeros(shape=(len(batch_protein_names), 700, 4))
        
        for batch_index, protein_name in enumerate(batch_protein_names):
            data_path = self.dataset_path + os.sep + "Rawdata" + os.sep + protein_name + os.sep + protein_name
            protein_features = []
            
            for feature in self.features:
                if self.features[feature]["add_feature"]:
                    with open(data_path + '_' + self.features[feature]["extension"] + ".npy", 'rb') as feature_file:
                        protein_features.append(np.nan_to_num(np.load(file=feature_file), nan=0.0))
            
            main_input[batch_index, :self.proteins_dict[protein_name]] = np.concatenate(protein_features, axis=-1)
            attention_mask[batch_index, self.proteins_dict[protein_name]:] = -np.inf
            position_ids[batch_index] = np.arange(700)
            weight_mask[batch_index, self.proteins_dict[protein_name]:] = 0
        
        return {"main_input": main_input, "attention_mask": attention_mask, "position_ids": position_ids}, {"Q8_output": Q8_labels, "Phi_Psi_output": phi_psi_labels}, weight_mask

def load_labels_train(dataset_path, proteins_dict, protein_names):
    Q8_labels = np.zeros(shape=(len(protein_names), 700, 8))
    phi_psi_labels = np.full(shape=(len(protein_names), 700, 4), fill_value=-500, dtype=np.float32)
    
    for index, protein_name in enumerate(protein_names):
        data_path = dataset_path + os.sep + "Rawdata" + os.sep + protein_name + os.sep + protein_name
        
        with open(data_path + "_ss8.npy", 'rb') as label_file:
            Q8_labels[index, :proteins_dict[protein_name]] = np.load(file=label_file)
        
        with open(data_path + "_phi.npy", 'rb') as label_file:
            phi_angles = np.load(file=label_file)
            phi_angles = np.reshape(phi_angles, newshape=(-1,))
        
        with open(data_path + "_psi.npy", 'rb') as label_file:
            psi_angles = np.load(file=label_file)
            psi_angles = np.reshape(psi_angles, newshape=(-1,))
        
        phi_psi_labels[index, :proteins_dict[protein_name], 0] = np.sin(phi_angles * np.pi / 180)
        phi_psi_labels[index, :proteins_dict[protein_name], 1] = np.cos(phi_angles * np.pi / 180)
        phi_psi_labels[index, :proteins_dict[protein_name], 2] = np.sin(psi_angles * np.pi / 180)
        phi_psi_labels[index, :proteins_dict[protein_name], 3] = np.cos(psi_angles * np.pi / 180)
        
        phi_psi_labels[index, np.where(phi_angles == -500), :] = -500
        phi_psi_labels[index, np.where(psi_angles == -500), :] = -500
        phi_psi_labels[index, 0, :] = -500
        phi_psi_labels[index, proteins_dict[protein_name] - 1, :] = -500
    
    return Q8_labels, phi_psi_labels

In [ ]:
def load_labels_test(dataset_path, proteins_dict, protein_names):
    Q8_labels = np.zeros(shape=(len(protein_names), 700, 8))
    phi_psi_labels = np.full(shape=(len(protein_names), 700, 4), fill_value=-500, dtype=np.float32)
    
    for index, protein_name in enumerate(protein_names):
        data_path = dataset_path + os.sep + "Rawdata" + os.sep + protein_name + os.sep + protein_name
        
        with open(data_path + "_ss8.npy", 'rb') as label_file:
            Q8_labels[index, :proteins_dict[protein_name]] = np.load(file=label_file)
        
        with open(data_path + "_phi.npy", 'rb') as label_file:
            phi_angles = np.load(file=label_file)
            phi_angles = np.reshape(phi_angles, newshape=(-1,))
        
        with open(data_path + "_psi.npy", 'rb') as label_file:
            psi_angles = np.load(file=label_file)
            psi_angles = np.reshape(psi_angles, newshape=(-1,))
        
        phi_psi_labels[index, :proteins_dict[protein_name], 0] = np.sin(phi_angles * np.pi / 180)
        phi_psi_labels[index, :proteins_dict[protein_name], 1] = np.cos(phi_angles * np.pi / 180)
        phi_psi_labels[index, :proteins_dict[protein_name], 2] = np.sin(psi_angles * np.pi / 180)
        phi_psi_labels[index, :proteins_dict[protein_name], 3] = np.cos(psi_angles * np.pi / 180)
        
        phi_psi_labels[index, np.where(phi_angles == -500), :2] = -500
        phi_psi_labels[index, np.where(psi_angles == -500), 2:] = -500
        phi_psi_labels[index, 0, :2] = -500
        phi_psi_labels[index, proteins_dict[protein_name] - 1, 2:] = -500
    
    return Q8_labels, phi_psi_labels

### Inference Parameters and Base Models Configuration

In [ ]:
test_set_name = "TEST2018"
test_set_path = "../datasets/SPOT-1D/Features" + os.sep + test_set_name
test_batch_size = 1

base_models = [
    "SAINT_Martin_Basic2_PSSM_HHM_PCP", 
    "SAINT_Martin_Basic2_PSSM_HHM_PCP_Win10", 
    "SAINT_Martin_Basic1_PT256_PSSM_HHM_PCP", 
    "SAINT_Martin_Basic1_PT256_PSSM_HHM_PCP_Win10", 
    "SAINT_Martin_Basic1_PT256_PSSM_HHM_PCP_Win20", 
    "SAINT_Martin_Basic1_PT256_PSSM_HHM_PCP_Win50", 
    "SAINT_Martin_Residual1_PT243_PSSM_HHM_PCP", 
    "SAINT_Martin_Residual1_PT227_PSSM_HHM_PCP_Win10"
]

ensemble_name = "Ensemble_SAINT-Angle_8"
compute_performance_metrics = False
generate_predicted_labels = True

custom_objects = {
    "CustomLayer": CustomLayer, 
    "backend": backend, 
    "get_shape_list": get_shape_list, 
    "custom_categorical_cross_entropy": custom_categorical_cross_entropy, 
    "average_accuracy": average_accuracy, 
    "total_accuracy": total_accuracy, 
    "total_correct_prediction": total_correct_prediction, 
    "total_tse": total_tse, 
    "mean_tse": mean_tse, 
    "total_mse": total_mse, 
    "mean_mse": mean_mse, 
    "mean_mae": mean_mae, 
    "phi_mae": phi_mae, 
    "psi_mae": psi_mae, 
    "mean_sae": mean_sae, 
    "phi_sae": phi_sae, 
    "psi_sae": psi_sae
}

### Making Predictions with Base Models and Ensemble

In [ ]:
test_proteins_dict, test_protein_names = None, None
Q8_predictions, phi_psi_predictions = {}, {}

inference_start_time = time.time()

for base_model in base_models:
    print(f"Making prediction with {base_model}...")
    
    with open("../Best_Model" + os.sep + base_model + "_args.txt", 'r') as args_file:
        args = ast.literal_eval(args_file.read())
    
    model = load_model(filepath=args["best_model_path"], custom_objects=custom_objects)
    test_dataloader = TestDataLoader(dataset_path=test_set_path, dataset_name=test_set_name, features=args["features"], num_features=args["num_features"], batch_size=test_batch_size)
    
    if test_proteins_dict is None:
        test_proteins_dict = test_dataloader.proteins_dict
    
    if test_protein_names is None:
        test_protein_names = test_dataloader.protein_names
    
    for global_protein_name, local_protein_name in zip(test_protein_names, test_dataloader.protein_names):
        assert global_protein_name == local_protein_name
    
    Q8_prediction, phi_psi_prediction = model.predict(x=test_dataloader)
    Q8_predictions[base_model], phi_psi_predictions[base_model] = Q8_prediction, phi_psi_prediction

if compute_performance_metrics:
    Q8_labels_train, phi_psi_labels_train = load_labels_train(dataset_path=test_set_path, proteins_dict=test_proteins_dict, protein_names=test_protein_names)
    Q8_labels_test, phi_psi_labels_test = load_labels_test(dataset_path=test_set_path, proteins_dict=test_proteins_dict, protein_names=test_protein_names)

ensemble_Q8_prediction, ensemble_phi_psi_prediction = None, None

for base_model in base_models:
    if ensemble_Q8_prediction is None:
        ensemble_Q8_prediction = Q8_predictions[base_model]
    else:
        ensemble_Q8_prediction = ensemble_Q8_prediction + Q8_predictions[base_model]
    
    if ensemble_phi_psi_prediction is None:
        ensemble_phi_psi_prediction = phi_psi_predictions[base_model]
    else:
        ensemble_phi_psi_prediction = ensemble_phi_psi_prediction + phi_psi_predictions[base_model]
    
    if compute_performance_metrics:
        average_accuracy_train = average_accuracy_numpy(Q8_labels_train, Q8_predictions[base_model])
        total_accuracy_train = total_accuracy_numpy(Q8_labels_train, Q8_predictions[base_model])
        total_correct_prediction_train = total_correct_prediction_numpy(Q8_labels_train, Q8_predictions[base_model])
        
        mean_mae_train = mean_mae_numpy_train(phi_psi_labels_train, phi_psi_predictions[base_model])
        phi_mae_train = phi_mae_numpy_train(phi_psi_labels_train, phi_psi_predictions[base_model])
        psi_mae_train = psi_mae_numpy_train(phi_psi_labels_train, phi_psi_predictions[base_model])
        
        average_accuracy_test = average_accuracy_numpy(Q8_labels_test, Q8_predictions[base_model])
        total_accuracy_test = total_accuracy_numpy(Q8_labels_test, Q8_predictions[base_model])
        total_correct_prediction_test = total_correct_prediction_numpy(Q8_labels_test, Q8_predictions[base_model])
        
        mean_mae_test = mean_mae_numpy_test(phi_psi_labels_test, phi_psi_predictions[base_model])
        phi_mae_test = phi_mae_numpy_test(phi_psi_labels_test, phi_psi_predictions[base_model])
        psi_mae_test = psi_mae_numpy_test(phi_psi_labels_test, phi_psi_predictions[base_model])
        
        print(f"\n{test_set_name} -- {base_model}:-")
        print(f"Avg Accuracy [Train/Test]: {average_accuracy_train:.5f}/{average_accuracy_test:.5f}")
        print(f"Total Accuracy [Train/Test]: {total_accuracy_train:.5f}/{total_accuracy_test:.5f}")
        print(f"Total Correct Prediction [Train/Test]: {total_correct_prediction_train:.2f}/{total_correct_prediction_test:.2f}")
        print(f"Mean MAE [Train/Test]: {mean_mae_train:.4f}/{mean_mae_test:.4f}")
        print(f"Phi MAE [Train/Test]: {phi_mae_train:.4f}/{phi_mae_test:.4f}")
        print(f"Psi MAE [Train/Test]: {psi_mae_train:.4f}/{psi_mae_test:.4f}")

if len(base_models) > 1:
    ensemble_Q8_prediction = softmax(ensemble_Q8_prediction, axis=-1)
    ensemble_phi_psi_prediction = ensemble_phi_psi_prediction / len(base_models)
    
    if compute_performance_metrics:
        average_accuracy_train = average_accuracy_numpy(Q8_labels_train, ensemble_Q8_prediction)
        total_accuracy_train = total_accuracy_numpy(Q8_labels_train, ensemble_Q8_prediction)
        total_correct_prediction_train = total_correct_prediction_numpy(Q8_labels_train, ensemble_Q8_prediction)
        
        mean_mae_train = mean_mae_numpy_train(phi_psi_labels_train, ensemble_phi_psi_prediction)
        phi_mae_train = phi_mae_numpy_train(phi_psi_labels_train, ensemble_phi_psi_prediction)
        psi_mae_train = psi_mae_numpy_train(phi_psi_labels_train, ensemble_phi_psi_prediction)
        
        average_accuracy_test = average_accuracy_numpy(Q8_labels_test, ensemble_Q8_prediction)
        total_accuracy_test = total_accuracy_numpy(Q8_labels_test, ensemble_Q8_prediction)
        total_correct_prediction_test = total_correct_prediction_numpy(Q8_labels_test, ensemble_Q8_prediction)
        
        mean_mae_test = mean_mae_numpy_test(phi_psi_labels_test, ensemble_phi_psi_prediction)
        phi_mae_test = phi_mae_numpy_test(phi_psi_labels_test, ensemble_phi_psi_prediction)
        psi_mae_test = psi_mae_numpy_test(phi_psi_labels_test, ensemble_phi_psi_prediction)
        
        print(f"\n{test_set_name} -- Ensemble:-")
        print(f"Avg Accuracy [Train/Test]: {average_accuracy_train:.5f}/{average_accuracy_test:.5f}")
        print(f"Total Accuracy [Train/Test]: {total_accuracy_train:.5f}/{total_accuracy_test:.5f}")
        print(f"Total Correct Prediction [Train/Test]: {total_correct_prediction_train:.2f}/{total_correct_prediction_test:.2f}")
        print(f"Mean MAE [Train/Test]: {mean_mae_train:.4f}/{mean_mae_test:.4f}")
        print(f"Phi MAE [Train/Test]: {phi_mae_train:.4f}/{phi_mae_test:.4f}")
        print(f"Psi MAE [Train/Test]: {psi_mae_train:.4f}/{psi_mae_test:.4f}")
    
    base_models.append(ensemble_name)
    Q8_predictions[ensemble_name] = ensemble_Q8_prediction
    phi_psi_predictions[ensemble_name] = ensemble_phi_psi_prediction

### Postprocessing and Saving Results

In [ ]:
if generate_predicted_labels:
    if not os.path.exists("../Outputs" + os.sep + test_set_name):
        os.makedirs("../Outputs" + os.sep + test_set_name)
    
    ss_s = ['G', 'H', 'I', 'B', 'E', 'S', 'T', 'C']
    
    for base_model in base_models:
        if not os.path.exists("../Outputs" + os.sep + test_set_name + os.sep + base_model):
            os.makedirs("../Outputs" + os.sep + test_set_name + os.sep + base_model)
        
        for index, protein_name in enumerate(test_protein_names):
            with open(f"{test_set_path}/Rawdata/{protein_name}/{protein_name}.fasta", 'r') as fasta_file:
                pseq = fasta_file.read().split('\n')[1]
            
            assert len(pseq) == test_proteins_dict[protein_name]
            
            Q8_prediction = Q8_predictions[base_model][index, :len(pseq)]
            phi_psi_prediction = phi_psi_predictions[base_model][index, :len(pseq)]
            
            sseq, max_indices = "", np.argmax(Q8_prediction, axis=-1)
            
            for max_index in max_indices:
                sseq = sseq + ss_s[max_index]
            
            phi_angles = np.arctan2(phi_psi_prediction[:, 0], phi_psi_prediction[:, 1]) * 180 / np.pi
            psi_angles = np.arctan2(phi_psi_prediction[:, 2], phi_psi_prediction[:, 3]) * 180 / np.pi
            
            phi_angles[0] = psi_angles[len(pseq) - 1] = -500
            
            assert len(pseq) == len(sseq) == phi_angles.shape[0] == psi_angles.shape[0]
            
            outputs = pd.DataFrame({"Amino Acid": list(pseq), "SS": list(sseq), "Phi": phi_angles, "Psi": psi_angles})
            outputs.to_csv(f"../Outputs/{test_set_name}/{base_model}/{protein_name}.csv", index=False)

inference_required_time = time.time() - inference_start_time

print(f"\nDone! ~ Inference and labels generation took {inference_required_time} seconds.")